# 1. Marwa Alifa Azkiya (202010370311481)
# 2. Fhara Elvina Pingky Nadya (202010370311483)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import Libraries
Ini adalah library yang dibutuhkan dalam pengolahan data.

In [2]:
def downsample_dataset(dataset, label_column, desired_samples):
    labels = dataset[label_column].unique()
    sampled_data = pd.DataFrame(columns=dataset.columns)

    for label in labels:
        label_data = dataset[dataset[label_column] == label]
        sampled_data = pd.concat([sampled_data, label_data.sample(n=desired_samples, random_state=42, replace = True)])
    
    return sampled_data

# Make function to downsample
Membuat fungsi untuk menggurangi dataset dengan downsampling dikarenakan jika data terlalu banyak tidak bisa diproses

In [3]:
dataset = pd.read_csv('twitter_MBTI.csv')

split = dataset['label'].apply(lambda x: pd.Series(list(x)))
split.columns= [f'dimension{i+1}' for i in range(split.shape[1])]

dataset = pd.concat([dataset, split], axis=1)

dataset

,Unnamed: 0,text,label,dimension1,dimension2,dimension3,dimension4
0,0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj,i,n,t,j
1,1,@Hispanthicckk Being you makes you look cute||...,intj,i,n,t,j
2,2,@Alshymi Les balles sont réelles et sont tirée...,intj,i,n,t,j
3,3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj,i,n,t,j
4,4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj,i,n,t,j
...,...,...,...,...,...,...,...
7806,7806,"@sobsjjun God,,pls take care 😕|||@sobsjjun Hir...",intp,i,n,t,p
7807,7807,@Ignis_02 wow last time i got intp https://t.c...,intp,i,n,t,p
7808,7808,@akupilled A 100%|||@akupilled That SOMEONE wi...,entp,e,n,t,p
7809,7809,If you’re #INTJ this one is for you | What is ...,infj,i,n,f,j


## Load and Split Label in Dataset
membagi kolom label yang berisi MBTI secara keseluruhan dibagi menjadi 4 dimensi sesuai dengan kategorinya, hal ini dilakukan untuk mencegah kebingungan machine learning saat melakukan klasifikasi

In [4]:
downsampled_dataset = downsample_dataset(dataset, 'label', 10)

## Use function downsample
Disini menggunakan 10 data set pada setiap masing masing label. Pada data terdapat 16 jenis label yang berarti data setelah didownsample adalah 160

In [5]:
# Memisahkan data menjadi training set dan test set
train_data, test_data, train_labels, test_labels = train_test_split(downsampled_dataset['text'], downsampled_dataset['dimension1'], test_size=0.2, random_state=42)
train_data2, test_data2, train_labels2, test_labels2 = train_test_split(downsampled_dataset['text'], downsampled_dataset['dimension2'], test_size=0.2, random_state=42)
train_data3, test_data3, train_labels3, test_labels3 = train_test_split(downsampled_dataset['text'], downsampled_dataset['dimension3'], test_size=0.2, random_state=42)
train_data4, test_data4, train_labels4, test_labels4 = train_test_split(downsampled_dataset['text'], downsampled_dataset['dimension4'], test_size=0.2, random_state=42)

## Splitting Dataset
Pada proses ini data akan dibagi menjadi 2 yaitu :

data train -> data untuk pembelajaran data test -> data untuk pengujian disini kami menggunakan test_size 0.2 dan random_state 42 yang berarti data akan dibagi menjadi 20% data test dan 80% data train serta diacak

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Initialize the tokenizer and BERT model for dimension1

In [7]:
train_encodings = tokenizer(list(train_data), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data), truncation=True, padding=True)

## Text tokenization

In [8]:
import torch
# Mendapatkan representasi vektor teks menggunakan BERT
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])

with torch.no_grad():
    train_outputs = model(train_input_ids, train_attention_mask)
    train_features = train_outputs[0][:, 0, :].numpy()  # Menggunakan output dari token [CLS] sebagai fitur
    
    test_outputs = model(test_input_ids, test_attention_mask)
    test_features = test_outputs[0][:, 0, :].numpy()

In [9]:
# Menginisialisasi dan melatih model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels)

# Memprediksi label pada data uji
predictions = rf_model.predict(test_features)

# E/I

In [10]:
# Menampilkan laporan klasifikasi
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           e       0.58      0.69      0.63        16
           i       0.62      0.50      0.55        16

    accuracy                           0.59        32
   macro avg       0.60      0.59      0.59        32
weighted avg       0.60      0.59      0.59        32



In [11]:
train_encodings = tokenizer(list(train_data2), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data2), truncation=True, padding=True)

## Initialize the tokenizer and BERT model for dimension2

In [12]:
import torch
# Mendapatkan representasi vektor teks menggunakan BERT
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])

with torch.no_grad():
    train_outputs = model(train_input_ids, train_attention_mask)
    train_features = train_outputs[0][:, 0, :].numpy()  # Menggunakan output dari token [CLS] sebagai fitur
    
    test_outputs = model(test_input_ids, test_attention_mask)
    test_features = test_outputs[0][:, 0, :].numpy()

In [13]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels2)

# Memprediksi label pada data uji
predictions = rf_model.predict(test_features)

In [14]:
# Menampilkan laporan klasifikasi
print(classification_report(test_labels2, predictions))

              precision    recall  f1-score   support

           n       0.55      0.38      0.44        16
           s       0.52      0.69      0.59        16

    accuracy                           0.53        32
   macro avg       0.53      0.53      0.52        32
weighted avg       0.53      0.53      0.52        32



In [15]:
train_encodings = tokenizer(list(train_data3), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data3), truncation=True, padding=True)

In [16]:
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])

with torch.no_grad():
    train_outputs = model(train_input_ids, train_attention_mask)
    train_features = train_outputs[0][:, 0, :].numpy()  # Menggunakan output dari token [CLS] sebagai fitur
    
    test_outputs = model(test_input_ids, test_attention_mask)
    test_features = test_outputs[0][:, 0, :].numpy()

In [17]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels3)

# Memprediksi label pada data uji
predictions = rf_model.predict(test_features)

In [18]:
# Menampilkan laporan klasifikasi
print(classification_report(test_labels3, predictions))

              precision    recall  f1-score   support

           f       0.43      1.00      0.60        12
           t       1.00      0.20      0.33        20

    accuracy                           0.50        32
   macro avg       0.71      0.60      0.47        32
weighted avg       0.79      0.50      0.43        32



In [19]:
train_encodings = tokenizer(list(train_data4), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data4), truncation=True, padding=True)

In [20]:
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])

with torch.no_grad():
    train_outputs = model(train_input_ids, train_attention_mask)
    train_features = train_outputs[0][:, 0, :].numpy()  # Menggunakan output dari token [CLS] sebagai fitur
    
    test_outputs = model(test_input_ids, test_attention_mask)
    test_features = test_outputs[0][:, 0, :].numpy()

In [21]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels4)

# Memprediksi label pada data uji
predictions = rf_model.predict(test_features)

In [22]:
# Menampilkan laporan klasifikasi
print(classification_report(test_labels4, predictions))

              precision    recall  f1-score   support

           j       0.47      0.69      0.56        13
           p       0.69      0.47      0.56        19

    accuracy                           0.56        32
   macro avg       0.58      0.58      0.56        32
weighted avg       0.60      0.56      0.56        32

